# Demo

In this demo, we show how a model trained with `train.py` on the SNLI dataset performs the Natural Language Inference task on which it was originally trained.

## Setup

In [1]:
import os
from pathlib import Path
import pickle
import numpy as np
import torch
from models import NLIModel
from data import SNLIDataModule
import nltk

# Download nltk prerequisite for tokenization if not available already
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

In [2]:
# The checkpoints and vocabulary have to be downloaded seperately, see README.md for further instructions
ENCODER_CHECKPOINT_PATH = Path("./checkpoints/best/unilstm-zero.ckpt")
CLASSIFIER_CHECKPOINT_PATH = Path("./checkpoints/best/unilstm-zero-classifier.ckpt")
VOCABULARY_PATH = Path("./cache/vocab-zero.pkl")

In [3]:
# Load 
print(f"Loading encoder from {ENCODER_CHECKPOINT_PATH}")
encoder = torch.load(ENCODER_CHECKPOINT_PATH)
encoder.eval()
print(f"Encoder loaded.")

# Load classifier
print(f"Loading classifier from {CLASSIFIER_CHECKPOINT_PATH}")
classifier = torch.load(CLASSIFIER_CHECKPOINT_PATH)
classifier.eval()
print(f"Classifier loaded.")

# Load vocab
print(f"Loading vocabulary from {VOCABULARY_PATH}")
with open(VOCABULARY_PATH, 'rb') as handle:
    vocab = pickle.load(handle)
print(f"Vocabulary loaded.")

# Create model with full pipeline for NLI inference
model = NLIModel(encoder=encoder, classifier=classifier)

Loading encoder from checkpoints\best\unilstm-zero.ckpt
Encoder loaded.
Loading classifier from checkpoints\best\unilstm-zero-classifier.ckpt
Classifier loaded.
Loading vocabulary from cache\vocab-zero.pkl
Vocabulary loaded.


c:\Development\Miniconda3\envs\atcs\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder'])`.
  rank_zero_warn(
c:\Development\Miniconda3\envs\atcs\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'classifier' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['classifier'])`.
  rank_zero_warn(


In [4]:
# Define a function for doing inference and printing results in a pretty way
def predict_example(nli_model, premise, hypothesis):
    label_meanings = {
        0: "Entailment",
        1: "Neutral",
        2: "Contradiction",
    }
    premise = premise.lower()
    hypothesis = hypothesis.lower()

    # Create batch containing the example in the form expected by the model
    premise_tensor = torch.stack([vocab.get_embedding(token) for token in nltk.word_tokenize(premise)], dim=0)
    hypothesis_tensor = torch.stack([vocab.get_embedding(token) for token in nltk.word_tokenize(hypothesis)], dim=0)

    premise_length = premise_tensor.shape[0]
    hypothesis_length = hypothesis_tensor.shape[0]

    example_batch = (
        (torch.nn.utils.rnn.pad_sequence([premise_tensor], batch_first=True), [premise_length]),
        (torch.nn.utils.rnn.pad_sequence([hypothesis_tensor], batch_first=True), [hypothesis_length]),
        None
    )

    logits = nli_model.forward(example_batch)[0]
    predicted_class = logits.argmax().item()
    print(f"The model predicts: {label_meanings[predicted_class]}")

## Simple example
In this section we demonstrate the usage of our models on a simple example. The premise and hypothesis can of course be modified.

In [5]:
premise = "It is snowing outside my house"
hypothesis = "Today is a great day to go swimming at the beach"
print("Correct label: Contradiction")
predict_example(model, premise, hypothesis)

Correct label: Contradiction
The model predicts: Contradiction


## Hard examples
Let's continue with some hard examples.

In [6]:
premise = "Two men sitting in the sun"
hypothesis = "Nobody is sitting in the shade"
print("Correct label: Neutral")
predict_example(model, premise, hypothesis)

Correct label: Neutral
The model predicts: Contradiction


In [7]:
premise = "A man is walking a dog"
hypothesis = "No cat is outside"
print("Correct label: Neutral")
predict_example(model, premise, hypothesis)

Correct label: Neutral
The model predicts: Contradiction


In these examples, the hypothesis contains a statement about something that sounds opposite of the premise (i.e. "sun"<->"shade" and "dog"<->"cat"), but the hypothesis is not clearly scoped. The model has difficulty predicting the correct relationship (neutral, in both cases) with certainty. It seems like the model assumes that the premise contains the entire truth. 

Interestingly, a similar cognitive bias occuring in humans called "What You See is All There Is" (WYSIATI) is described by Daniel Kahneman in Thinking, Fast and Slow.